# Finding the parameters of Lotka-Volterra equations with Evolution Strategies

The Lotka-Volterra equations, named after Alfred Lotka and Vito Volterra, are a mathematical model for understanding how the populations of predator and prey species can influence each other's growth and evolution over time. The equations consist of two coupled first-order differential equations, one for the predator population ($Y$) and one for the prey population ($X$):

$$  \frac{dX}{dt} = rX - aXY $$

$$  \frac{dY}{dt} = bXY -zY $$

Equations are defined by a set of parameters, such as $r$, $a$, $z$, and $b$, in the previous formulation.

The goal of this lab session is to find the parameters of a dynamic system defined by Lotka-Volterra equations leveraging **Evolution Strategies** (ES). We will consider having access to the trajectories corresponding to a single simulation.

First of all, let us import some useful modules and functions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.integrate import odeint #solver for a system of ordinary differential equations
import sklearn as sk

random.seed(0)
np.random.seed(0)

Now, we want to write the code to actually perform the simulation and get our trajectories.  Let's define a function that, given the current state, the time instant, and the parameters, returns the derivatives calculated by the Lotka-Volterra equations. We don't know who is the prey and the predator a priori, the sign is given by the parameter itself!

In [ ]:
# Define the Lotka-Volterra equations
def lotka_volterra(state, t, params):
    dxdt = params[0][0]*state[0] - params[0][1]*state[0]*state[1]
    dydt = params[1][0]*state[0]*state[1] - params[1][1]*state[1]
    return [dxdt, dydt]

Now we arbitrarily choose the initial conditions of our simulation, a set of parameters and the time vector.

In [ ]:
# Set the initial conditions and parameters
initial_conditions = [30, 15]  # Initial populations of species X and Y

real_params = np.array(
    [[0.1, 0.02],
    [0.01, 0.1]],
)

# Create a time vector
t = np.linspace(0, 100, 1000)

# Solve the differential equations using odeint
solution = odeint(lotka_volterra, initial_conditions, t, args=(real_params,))
# Extract the populations of each species
x, y = solution.T

In [ ]:
# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(t, x, label='Species X')
plt.plot(t, y, label='Species Y')
plt.xlabel('Time')
plt.ylabel('Population')
plt.legend()
plt.title('Lotka-Volterra Model with 2 Species')
plt.grid(True)
plt.show()

Now, let's assume we don't know the parameters that generated the previous trajectories. We aim to find them using ES!

Define a fitness function which suits the problem. Each individual is a set of parameters for the Lotka-Volterra equations and we want to get as close as possible to the actual one through the evolution.

The fitness will be the inverse of the RMSE for the individual wrt the actual target function where $T=\{t\in\mathbb{N}:0<=t<=1000\}$ and the initial conditions are $(X=30, Y=10)$.

$$\text{fit}(g)=\frac{\text{RMSE}(X_{real}, X_{pred}) + \text{RMSE}(Y_{real}, Y_{pred})}{2}$$

In [ ]:
# Fitness function
def fit(individual):
    xp, yp = odeint(lotka_volterra, initial_conditions, t, args=(individual,)).T
    fitness = (sk.metrics.root_mean_squared_error(x, xp) + sk.metrics.mean_squared_error(y, yp))/2
    return fitness

Define a class for our ES. It must implement the One-Fifth Rule for variance self-adaptation. Implement a strategy of your choice for recombination.

In [ ]:
def recombination(parents):
    return parents[random.randint(0, len(parents) - 1)]

In [ ]:
class EvolutionStrategy(object):

    def __init__(self, mu, lmb, ro, fit, rec, individual_shape, limits, plus=False, initial_variance=1, k=10, c=0.5):
        self.mu = mu
        self.lmb = lmb
        self.ro = ro
        self.plus = plus
        self.fit = fit
        self.rec = rec #recombination strategy
        self.individual_shape = individual_shape
        self.limits = limits
        self.pop = []
        self.variance = initial_variance
        self.k = k
        self.c = c
        self.k_counter = 0

        # Initialize the population
        lower, upper = self.limits
        for _ in range(self.lmb):
            individual = np.zeros(self.individual_shape)
            for i in range(self.individual_shape[0]):
                for j in range(self.individual_shape[1]):
                    individual[i,j] = random.uniform(lower, upper)
            self.pop.append(individual)
        self.best = min(self.pop, key=self.fit)
        self.best_fit=self.fit(self.best)

    def generation(self):
        # CODE HERE
        self.k_counter += 1
        self.truncated_selection()
        new_pop = []
        fitter = 0
        p_s = 0
        for i in range(0, self.lmb):
            # recombination + mutation
            old_ind = recombination(self.pop)
            new_ind = self.mutation(old_ind)
            if self.fit(new_ind) < self.fit(old_ind):
                fitter = fitter + 1
            new_pop.append(new_ind)
        p_s = fitter / len(new_pop)
        if self.k_counter % self.k == 0:
            # 1/5 rule
            self.variance = self.variance / self.c if p_s >= 0.20 else self.variance * self.c
        if self.plus:
            self.pop = self.pop + new_pop
        else:
            self.pop = new_pop
        candidate_best = min(self.pop, key=self.fit)
        if self.fit(self.best) > self.fit(candidate_best):
            self.best = candidate_best
            self.best_fit = self.fit(self.best)

    def mutation(self, individual):
        mutated_individual = np.zeros(self.individual_shape)
        lower, upper = self.limits
        for i in range(self.individual_shape[0]):
            for j in range(self.individual_shape[1]):
                mut = np.random.normal(loc=0, scale=self.variance)
                mutparam = individual[i,j] + mut
                mutated_individual[i,j] = mutparam if lower < mutparam < upper else upper if mutparam > 0 else lower
        return mutated_individual

    def truncated_selection(self):
        list.sort(self.pop, key=self.fit)
        self.pop = self.pop[0:self.mu]

    def __str__(self):
        s = f"Best individual: {self.best}\n"
        s += f"with fitness {self.fit(self.best)}"
        return s

Run the evolution and save the best fitness score for each generation.

In [ ]:
ES = EvolutionStrategy(50, 100, 2, fit, recombination, (2, 2), [-0.2, 0.2], k=20, plus=True)
history = [ES.best_fit]
n_gen = 300
for i in range(0, n_gen):
    ES.generation()
    history.append(ES.best_fit)

Plot and visualize the results

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history)
plt.ylabel('Fitness')
plt.xlabel('Generation')
plt.show()

In [ ]:
res_best = odeint(lotka_volterra, initial_conditions, t, args=(ES.best,))

# Extract the populations of each species
x, y = res_best.T

In [ ]:
# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(t, x, label='Species X')
plt.plot(t, y, label='Species Y')
plt.xlabel('Time')
plt.ylabel('Population')
plt.legend()
plt.title('Lotka-Volterra Model with 2 Species')
plt.grid(True)
plt.show()

In [ ]:
print(ES)